In [1]:
#!/usr/bin/env python

from random import choice
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
from config import GetProducerConfig

# if __name__ == '__main__':

config = GetProducerConfig()
#config['schema.registry.url'] = "http://pkc-12576z.us-west2.gcp.confluent.cloud:9092/schemas/ids/100001"
# Create Producer instance
producer = Producer(config)


In [68]:
def convert_(message,ctx):
    return dict(
        idade=message.idade,
        name=message.name,
        team=message.team)
schema_registry_conf = {'url': "https://@psrc-57o65q.us-west2.gcp.confluent.cloud",
                        #'basic.auth.credentials.source': 'USER_INFO',
                        'basic.auth.user.info': "IBL6S7JRHBD2CNKV:hhaEwDXbCZM7K8G3dl5VeXO7G9DJoG5AoaiT1/msX/s0oefT1CJVrajruFdM3jEA"
                        }
schema_registry_client = SchemaRegistryClient(schema_registry_conf)
schema = schema_registry_client.get_schema(100001)
avro_serializer = AvroSerializer(schema_registry_client,
                                     schema.schema_str,
                                     convert_)
string_serializer = StringSerializer('utf_8')


In [69]:
import requests
def delivery_callback(err, msg):
    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))



topic = "topic_0"
count = 0


In [70]:
class Message(object):

    def __init__(self, idade, name, team):
        self.name = name
        self.idade = idade
        self.team = team


In [75]:
from uuid import uuid4

message  = Message(20,"Isaias","ML")
producer.produce(topic=topic,
                             key=string_serializer(str(uuid4())),
                             value=avro_serializer(message, SerializationContext(topic, MessageField.VALUE)),
                             on_delivery=delivery_callback)
producer.poll(10000)
producer.flush()

User record b'8be9fc9d-5b2a-452c-9539-7987905f27a4' successfully produced to topic_0 [0] at offset 6


0